# 패키지 로드

In [134]:
from itertools import islice # for print head
import re # 정규표현식 모듈
from pandas import DataFrame as df # dataframe 을 활용하기 위함
import pandas as pd

from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

from operator import eq

# 1. 문서전처리

### 1.2 데이터 로드 (사용할 챕터만 선택)

In [135]:
# 사용할 챕터 로드
# 로드를 못한다면 pip install xlrd
chapterspath = "data/chapters.xlsx"
prepro = pd.read_excel(chapterspath)


# 사용할 line 만을 선택: 사용할 챕터 선택, 빈줄은 삭제 
use_chaps = [2,3,4,5,6,7,9,11,13,15,17]
p = re.compile("^\n$")
for i in range(0, len(prepro)):
    m = p.match(prepro["book"][i])
    
    # i번째 행이 빈 경우 삭제
    if m!=None:
        prepro = prepro.drop([i],0)
        continue
    
    # i번째 행이 사용할 챕터가 아닌 경우, 삭제
    if prepro["chapter"][i] not in use_chaps:
        prepro = prepro.drop([i],0)

# 행을 삭제함에따라 index 를 새로 바꿔줌
prepro = prepro.reset_index()

# 기존의 index 열은 삭제
prepro = prepro.drop('index', 1)

In [136]:
prepro

,book,linenumber,chapter
0,Chapter 2\n,74,2
1,Mr. Bennet was among the earliest of those who...,75,2
2,"“I hope Mr. Bingley will like it, Lizzy.”\n",77,2
3,“We are not in a way to know what Mr. Bingley ...,79,2
4,"“But you forget, mamma,” said Elizabeth, “that...",81,2
5,“I do not believe Mrs. Long will do any such t...,83,2
6,"“No more have I,” said Mr. Bennet; “and I am g...",85,2
7,"Mrs. Bennet deigned not to make any reply, but...",87,2
8,"“Don't keep coughing so, Kitty, for Heaven's s...",89,2
9,"“Kitty has no discretion in her coughs,” said ...",91,2


### 1.3 Tokenization

In [137]:
# 문서 토큰화
tokens_arr = []
chapter_arr = []
#for i in range(0, 50):
for i in range(0, len(prepro)):
    orig_line = prepro["book"][i] # 토큰화 대상 라인
    parse = orig_line.lower() # 소문자화
    
    # 토큰화 전에 정규식으로 삭제할 수 있는 표현 삭제(stop word를 일부 여기서 처리)
    # [Chapter 숫자]라고 적힌 단어를 삭제
    parse = re.sub('chapter \d(\d)?', '', parse)
    # , 등 특수문자를 삭제
    parse = re.sub('[.,?!\n“”:;]', '', parse)
    
    # 위의 삭제할 문자들을 삭제하고 난 후 남는 문자열이 없으면 처리를 하지 않음
    if eq(parse, ""):
        continue
    
    # 토큰화
    token_lines = parse.split(' ')
    for j in range(len(token_lines)):
        tokens_arr.append(token_lines[j])
        chapter_arr.append(prepro["chapter"][i])
        
#print(len(tokens_arr))
#print(len(chapter_arr))
#print(tokens_arr)
#print(chapter_arr)


In [138]:
# token 정보를 저장할 데이터 프레임
tokens = df(data = {'token':tokens_arr, 'chapter':chapter_arr})
tokens

,chapter,token
0,2,mr
1,2,bennet
2,2,was
3,2,among
4,2,the
5,2,earliest
6,2,of
7,2,those
8,2,who
9,2,waited


### 1.3 Stopwords

In [139]:
# nltk stopword 를 다운로드
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kkm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [140]:
# stop words 로드
stop = set(stopwords.words('english'))

In [141]:
# stop words 에 해당하는 token 모두 삭제
# (연구 한계) stemming 하지 않았으므로, stop words 에서 일부 삭제되지 않은 단어가 있을 것임.
for i in range(0, len(tokens)):    
    # stop word 에 해당하는 경우 삭제
    if tokens["token"][i] in stop:
        tokens = tokens.drop([i],0)
        
# 행을 삭제함에따라 index 를 새로 바꿔줌
tokens = tokens.reset_index()

# 기존의 index 열은 삭제
tokens = tokens.drop('index', 1)

In [142]:
tokens

,chapter,token
0,2,mr
1,2,bennet
2,2,among
3,2,earliest
4,2,waited
5,2,mr
6,2,bingley
7,2,always
8,2,intended
9,2,visit


In [143]:
import pandas as pd
writer = pd.ExcelWriter('data/prerocessed.xlsx')
tokens.to_excel(writer,'Sheet1')
writer.save()